# Anomalous Financial Transaction Detection

본 대회의 과제는 금융 거래 데이터에서 **이상 거래를 탐지하는 기능**을 개선하고 활용도를 높이는 분류 AI모델을 개발하는 것입니다. 

특히, 클래스 불균형 문제를 해결하기 위해 오픈소스 생성형 AI 모델을 활용하여 부족한 클래스의 데이터를 보완하고, 이를 통해 분류 모델의 성능을 향상시키는 것이 핵심 목표입니다. 

이러한 접근을 통해 금융보안에 특화된 데이터 분석 및 활용 역량을 강화하여 전문 인력을 양성하고, 금융권의 AI 활용 어려움에 따른 해결 방안을 함께 모색하며 금융 산업의 AI 활용 활성화를 지원하는 것을 목표로 합니다.

# Import Library

In [1]:
# pip install sdv

In [2]:
# 제출 파일 생성 관련
import os
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

# 머신러닝 전처리
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# 머신러닝 모델
import xgboost as xgb

# 합성 데이터 생성
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 생성 🏭

# Load Data

In [3]:
train_all = pd.read_csv("train.csv")
test_all = pd.read_csv("test.csv")

In [4]:
train = train_all.drop(columns="ID")

In [5]:
train.shape

(120000, 63)

In [6]:
train.head()

,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,Customer_rooting_jailbreak_indicator,Customer_mobile_roaming_indicator,Customer_VPN_Indicator,Customer_loan_type,Customer_flag_terminal_malicious_behavior_1,Customer_flag_terminal_malicious_behavior_2,Customer_flag_terminal_malicious_behavior_3,Customer_flag_terminal_malicious_behavior_4,Customer_flag_terminal_malicious_behavior_5,Customer_flag_terminal_malicious_behavior_6,Customer_inquery_atm_limit,Customer_increase_atm_limit,Account_account_number,Account_account_type,Account_creation_datetime,Account_initial_balance,Account_balance,Account_indicator_release_limit_excess,Account_amount_daily_limit,Account_indicator_Openbanking,Account_remaining_amount_daily_limit_exceeded,Account_release_suspention,Account_one_month_max_amount,Account_one_month_std_dev,Account_dawn_one_month_max_amount,Account_dawn_one_month_std_dev,Transaction_Datetime,Transaction_Amount,Channel,Operating_System,Error_Code,Transaction_Failure_Status,Type_General_Automatic,IP_Address,MAC_Address,Access_Medium,Location,Recipient_Account_Number,Transaction_num_connection_failure,Another_Person_Account,Distance,Time_difference,Unused_terminal_status,Last_atm_transaction_datetime,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Fraud_Type,Transaction_resumed_date
0,1980,male,이상호,BJWQxd-WBASPLJ,2003-01-06 18:38:01,B,0,1,0,1,0,0,0,a,0,0,0,0,0,0,0,0,oVZASOzgcm,c,2003-01-22 23:38:48,10390513,11270513,0,2000000,1,2000000,1,10000,0,0,0,2003-01-25 22:20:34,10000,internet,Others,a,0,general,171.237.22.26,44:b3:37:b1:2e:ce,b,강원도 고성군 죽왕면 38.354486 128.509098,zCoyEcbmDU,0,1,382.666923,0 days 02:53:50,1,2003-01-22 23:38:48,2003-01-22 23:38:48,1,1,1,0,0,0,m,2003-01-22 23:38:48
1,1964,male,박상철,kurCwX-odPUXEt,2003-01-07 16:40:44,C,0,1,0,0,0,0,0,a,0,0,0,0,0,0,0,0,ggkzgSNcaL,b,2003-01-19 21:29:08,10510556,9243706,0,1000000,0,1000000,1,25160000,12777419,25160000,12777419,2003-01-31 02:22:42,-25160000,Others,Windows,a,0,general,140.12.178.172,dc:85:65:a4:6c:2e,a,경상북도 영주시 이산면 36.824931 128.65353,WdNxDkGogG,0,1,59.491583,0 days 01:07:33,0,2003-01-21 21:29:08,2003-01-31 00:19:46,0,1,0,0,0,0,m,2003-01-19 21:29:08
2,1982,female,조옥자,OiERQa-CTXBoaX,2003-01-11 14:08:36,B,1,1,0,0,0,0,0,c,0,0,0,0,0,0,0,1,cPZYJzhMdy,d,2003-01-31 07:13:28,9080361,30778870,1,50000000,0,1000000,1,1250000,0,0,0,2003-01-31 10:20:12,20130000,Others,Others,a,0,general,28.184.85.32,bc:8b:6c:31:63:24,b,경상북도 안동시 길안면 36.483284 128.925046,JTQVmkaxGi,0,1,108.671532,0 days 00:52:59,1,2003-01-31 07:13:28,2003-01-31 07:13:28,0,0,1,1,1,0,m,2003-01-31 07:13:28
3,1982,female,조옥자,OiERQa-CTXBoaX,2003-01-11 14:08:36,B,1,1,1,0,0,0,0,c,0,0,0,0,0,0,1,1,cPZYJzhMdy,d,2003-01-31 07:13:28,9080361,35268870,0,2000000,1,1000000,1,2270000,721248,0,0,2003-01-31 13:14:01,24620000,Others,Windows,a,0,general,199.110.103.73,24:21:27:a7:bd:44,a,경상북도 의성군 다인면 36.489449 128.389484,FhhrvwJSLj,2,1,284.327590,0 days 01:24:05,1,2003-01-31 11:49:56,2003-01-31 07:13:28,1,1,0,0,0,0,m,2003-01-31 07:13:28
4,1982,female,조옥자,OiERQa-CTXBoaX,2003-01-11 14:08:36,B,1,1,1,0,0,0,0,c,0,0,0,1,0,0,1,1,cPZYJzhMdy,d,2003-01-31 07:13:28,26748870,10618870,0,2000000,1,970000,1,2270000,1121487,0,0,2003-01-31 15:22:26,-30000,mobile,Android,a,0,general,193.29.112.107,cc:7b:db:54:4f:ec,b,경상남도 밀양시 초동면 35.384189 128.660754,ylRXiwruID,2,1,244.587744,0 days 01:43:29,1,2003-01-31 11:49:56,2003-01-31 07:13:28,1,0,0,1,1,0,m,2003-01-31 07:13:28


In [17]:
'''
(*) 리더보드 산식 중 생성데이터의 익명성(TCAP)채점을 위해 각 클래스 별로 1000개의 생성데이터가 반드시 필요합니다.
(*) 본 베이스 라인에서는 "Fraud_Type" 13종류에 대해 1000개씩 , 총 13,000개의 데이터를 생성할 예정입니다.
(*) 분류 모델 성능 개선을 위해 생성 데이터를 활용하는 것에는 생성 데이터의 Row 개수에 제한이 없습니다. 단, 리더보드 평가를 위해 제출을 하는 생성 데이터 프레임은 익명성(TCAP) 평가를 위함이며, 위의 조건을 갖춘 생성 데이터를 제출해야합니다.
'''
N_CLS_PER_GEN = 1000
N_CLS_PER_GEN_2 = 20

## 제출용 합성데이터

In [16]:
from ctgan import CTGAN 
import pandas as pd
from tqdm import tqdm
import numpy as np
from scipy import stats
from sdv.metadata import SingleTableMetadata

# 이상치 처리 함수
def handle_outliers(series, n_std=3):
    mean = series.mean()
    std = series.std()
    z_scores = np.abs(stats.zscore(series))
    return series.mask(z_scores > n_std, mean)

# Time_difference 컬럼을 총 초로 변환 및 이상치 처리
train['Time_difference_seconds'] = pd.to_timedelta(train['Time_difference']).dt.total_seconds()
train['Time_difference_seconds'] = handle_outliers(train['Time_difference_seconds'])

# 모든 Fraud_Type 목록 생성
fraud_types = train['Fraud_Type'].unique()

# 모든 합성 데이터를 저장할 DataFrame 초기화
all_synthetic_data = pd.DataFrame()

N_SAMPLE = 100

# 각 Fraud_Type에 대해 합성 데이터 생성 및 저장
for fraud_type in tqdm(fraud_types):
    
    # 해당 Fraud_Type에 대한 서브셋 생성
    subset = train[train["Fraud_Type"] == fraud_type]

    # 모든 Fraud_Type에 대해 100개씩 샘플링
    subset = subset.sample(n=N_SAMPLE, random_state=42)
    
    # Time_difference 열 제외 (초 단위로 변환된 컬럼만 사용)
    subset = subset.drop('Time_difference', axis=1)
    
    # 메타데이터 생성 및 모델 학습
    metadata = SingleTableMetadata()

    metadata.detect_from_dataframe(subset)
    metadata.set_primary_key(None)

    # 데이터 타입 설정
    column_sdtypes = {
        'Account_initial_balance': 'numerical',
        'Account_balance': 'numerical',
        'Customer_identification_number': 'categorical',  
        'Customer_personal_identifier': 'categorical',
        'Account_account_number': 'categorical',
        'IP_Address': 'ipv4_address',  
        'Location': 'categorical',
        'Recipient_Account_Number': 'categorical',
        'Fraud_Type': 'categorical',
        'Time_difference_seconds': 'numerical',
        'Customer_Birthyear': 'numerical'
    }

    # 각 컬럼에 대해 데이터 타입 설정
    for column, sdtype in column_sdtypes.items():
        metadata.update_column(
            column_name=column,
            sdtype=sdtype
        )
        
    synthesizer = CTGANSynthesizer(
                            metadata,
                            epochs=1000
                        )
    synthesizer.fit(subset)

    synthetic_subset = synthesizer.sample(num_rows=N_CLS_PER_GEN)
    
    # 생성된 Time_difference_seconds의 이상치 처리
    synthetic_subset['Time_difference_seconds'] = handle_outliers(synthetic_subset['Time_difference_seconds'])
    
    # Time_difference_seconds를 다시 timedelta로 변환
    synthetic_subset['Time_difference'] = pd.to_timedelta(synthetic_subset['Time_difference_seconds'], unit='s')
    
    # Time_difference_seconds 컬럼 제거
    synthetic_subset = synthetic_subset.drop('Time_difference_seconds', axis=1)
    
    # 생성된 데이터를 all_synthetic_data에 추가
    all_synthetic_data = pd.concat([all_synthetic_data, synthetic_subset], ignore_index=True)
# 최종 결과 확인
print("\nFinal All Synthetic Data Shape:", all_synthetic_data.shape)


100%|██████████| 13/13 [15:25<00:00, 71.20s/it]


Final All Synthetic Data Shape: (13000, 63)


## 성능용 합성데이터

CTGAN

In [18]:
# from ctgan import CTGAN  # CTAB-GAN을 사용
# import pandas as pd
# from tqdm import tqdm


# # 모든 합성 데이터를 저장할 DataFrame 초기화
# all_synthetic_data_ctgan = pd.DataFrame()

# N_SAMPLE = 100

# # 각 Fraud_Type에 대해 합성 데이터 생성 및 저장
# for fraud_type in tqdm(fraud_types):
    
#     # 해당 Fraud_Type에 대한 서브셋 생성
#     subset = train[train["Fraud_Type"] == fraud_type]

#     # 모든 Fraud_Type에 대해 100개씩 샘플링
#     subset = subset.sample(n=N_SAMPLE, random_state=42)
    
#     # Time_difference 열 제외 (초 단위로 변환된 컬럼만 사용)
#     subset = subset.drop('Time_difference', axis=1)
    
#     # 메타데이터 생성 및 모델 학습
#     metadata = SingleTableMetadata()

#     metadata.detect_from_dataframe(subset)
#     metadata.set_primary_key(None)

#     # 데이터 타입 설정
#     column_sdtypes = {
#         'Account_initial_balance': 'numerical',
#         'Account_balance': 'numerical',
#         'Customer_identification_number': 'categorical',  
#         'Customer_personal_identifier': 'categorical',
#         'Account_account_number': 'categorical',
#         'IP_Address': 'ipv4_address',  
#         'Location': 'categorical',
#         'Recipient_Account_Number': 'categorical',
#         'Fraud_Type': 'categorical',
#         'Time_difference_seconds': 'numerical',
#         'Customer_Birthyear': 'numerical'
#     }

#     # 각 컬럼에 대해 데이터 타입 설정
#     for column, sdtype in column_sdtypes.items():
#         metadata.update_column(
#             column_name=column,
#             sdtype=sdtype
#         )
        
#     synthesizer = CTGANSynthesizer(
#                             metadata,
#                             epochs=100
#                         )
#     synthesizer.fit(subset)

#     synthetic_subset = synthesizer.sample(num_rows=N_CLS_PER_GEN_2) #
    
#     # 생성된 Time_difference_seconds의 이상치 처리
#     synthetic_subset['Time_difference_seconds'] = handle_outliers(synthetic_subset['Time_difference_seconds'])
    
#     # Time_difference_seconds를 다시 timedelta로 변환
#     synthetic_subset['Time_difference'] = pd.to_timedelta(synthetic_subset['Time_difference_seconds'], unit='s')
    
#     # Time_difference_seconds 컬럼 제거
#     synthetic_subset = synthetic_subset.drop('Time_difference_seconds', axis=1)
    
#     # 생성된 데이터를 all_synthetic_data에 추가
#     all_synthetic_data_ctgan = pd.concat([all_synthetic_data_ctgan, synthetic_subset], ignore_index=True)
# # 최종 결과 확인
# print("\nFinal All Synthetic Data ctgan Shape:", all_synthetic_data_ctgan.shape)


  0%|          | 0/13 [00:06<?, ?it/s]


KeyboardInterrupt: 

## 원본 데이터와 concat

In [19]:
origin_train = train_all.drop(columns="ID")
# train_total = pd.concat([origin_train, all_synthetic_data_ctgan])
train_total = origin_train.copy()
train_total.shape

(120000, 63)

# Data Preprocessing 1 : Select x, y

In [20]:
train_x = train_total.drop(columns=['Fraud_Type'])
train_y = train_total['Fraud_Type']

test_x = test_all.drop(columns=['ID'])

# Data Preprocessing 2 : 범주형 변수 인코딩

In [21]:
le_subclass = LabelEncoder()
train_y_encoded = le_subclass.fit_transform(train_y)

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

원래 레이블: a, 변환된 숫자: 0
원래 레이블: b, 변환된 숫자: 1
원래 레이블: c, 변환된 숫자: 2
원래 레이블: d, 변환된 숫자: 3
원래 레이블: e, 변환된 숫자: 4
원래 레이블: f, 변환된 숫자: 5
원래 레이블: g, 변환된 숫자: 6
원래 레이블: h, 변환된 숫자: 7
원래 레이블: i, 변환된 숫자: 8
원래 레이블: j, 변환된 숫자: 9
원래 레이블: k, 변환된 숫자: 10
원래 레이블: l, 변환된 숫자: 11
원래 레이블: m, 변환된 숫자: 12


In [22]:
# train_x
# 'Time_difference' 열을 문자열로 변환
train_x['Time_difference'] = train_x['Time_difference'].astype(str)

# 범주형 변수 인코딩
categorical_columns = train_x.select_dtypes(include=['object', 'category']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# 훈련 데이터 인코딩
train_x_encoded = train_x.copy()
train_x_encoded[categorical_columns] = ordinal_encoder.fit_transform(train_x[categorical_columns])

In [23]:
train_x_encoded['Customer_Total_Authentication_Changes'] = train_x_encoded[
    ['Customer_flag_change_of_authentication_1', 'Customer_flag_change_of_authentication_2', 
     'Customer_flag_change_of_authentication_3', 'Customer_flag_change_of_authentication_4']
].sum(axis=1)

train_x_encoded['Customer_Total_Malicious_Behavior_Flags'] = train_x_encoded[
    ['Customer_flag_terminal_malicious_behavior_1', 'Customer_flag_terminal_malicious_behavior_2', 
     'Customer_flag_terminal_malicious_behavior_3', 'Customer_flag_terminal_malicious_behavior_4', 
     'Customer_flag_terminal_malicious_behavior_5', 'Customer_flag_terminal_malicious_behavior_6']
].sum(axis=1)

train_x_encoded['Daily_Usage_Ratio'] = train_x_encoded['Account_amount_daily_limit'] / train_x_encoded['Account_remaining_amount_daily_limit_exceeded']

train_x_encoded['Transaction_Failure_Rate'] = train_x_encoded['Transaction_Failure_Status'].mean()


train_x_encoded['ATM_Limit_Increased'] = (train_x_encoded['Customer_increase_atm_limit'] > 0).astype(int)

train_x_encoded['Transaction_Weekday'] = pd.to_datetime(train_x_encoded['Transaction_Datetime']).dt.weekday


In [24]:
# 특성 순서 저장
feature_order = train_x_encoded.columns.tolist()

In [25]:
train_x_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)
train_x_encoded.fillna(train_x_encoded.mean(), inplace=True)

# Model Define

In [26]:
# import optuna
# import xgboost as xgb
# from sklearn.metrics import f1_score
# from sklearn.model_selection import train_test_split

# # 데이터 분할
# train_x, valid_x, train_y, valid_y = train_test_split(train_x_encoded, train_y_encoded, test_size=0.2, random_state=42)

# def objective(trial):
#     # 최적화할 하이퍼파라미터 범위 설정
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 50, 150),
#         'max_depth': trial.suggest_int('max_depth', 3, 7),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
#         'subsample': trial.suggest_float('subsample', 0.6, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#         'gamma': trial.suggest_float('gamma', 0, 10),
#         'random_state': 42,
#         'eval_metric': 'mlogloss',
#     }

#     # 모델 정의 및 학습
#     model = xgb.XGBClassifier(**param)
#     model.fit(train_x[feature_order], train_y, 
#               eval_set=[(valid_x[feature_order], valid_y)], 
#               early_stopping_rounds=10, 
#               verbose=False)
    
#     # 예측 및 Macro F1 계산
#     preds = model.predict(valid_x[feature_order])
#     macro_f1 = f1_score(valid_y, preds, average='macro')
    
#     return macro_f1


# # Optuna 스터디 생성
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=10)

# trial = study.best_trial

# # 최적 하이퍼파라미터로 모델 학습
# best_params = trial.params
# best_model = xgb.XGBClassifier(**best_params, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
# best_model.fit(train_x_encoded[feature_order], train_y_encoded)


In [27]:
# 테스트 데이터 인코딩
test_x_encoded = test_x.copy()
test_x_encoded[categorical_columns] = ordinal_encoder.transform(test_x[categorical_columns])

In [28]:
test_x_encoded['Customer_Total_Authentication_Changes'] = test_x_encoded[
    ['Customer_flag_change_of_authentication_1', 'Customer_flag_change_of_authentication_2', 
     'Customer_flag_change_of_authentication_3', 'Customer_flag_change_of_authentication_4']
].sum(axis=1)

test_x_encoded['Customer_Total_Malicious_Behavior_Flags'] = test_x_encoded[
    ['Customer_flag_terminal_malicious_behavior_1', 'Customer_flag_terminal_malicious_behavior_2', 
     'Customer_flag_terminal_malicious_behavior_3', 'Customer_flag_terminal_malicious_behavior_4', 
     'Customer_flag_terminal_malicious_behavior_5', 'Customer_flag_terminal_malicious_behavior_6']
].sum(axis=1)

test_x_encoded['Daily_Usage_Ratio'] = test_x_encoded['Account_amount_daily_limit'] / test_x_encoded['Account_remaining_amount_daily_limit_exceeded']

test_x_encoded['Transaction_Failure_Rate'] = test_x_encoded['Transaction_Failure_Status'].mean()


test_x_encoded['ATM_Limit_Increased'] = (test_x_encoded['Customer_increase_atm_limit'] > 0).astype(int)

test_x_encoded['Transaction_Weekday'] = pd.to_datetime(test_x_encoded['Transaction_Datetime']).dt.weekday

test_x_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)
test_x_encoded.fillna(test_x_encoded.mean(), inplace=True)

In [29]:
# 특성 순서 맞추기 및 데이터 타입 일치
test_x_encoded = test_x_encoded[feature_order]
# test_x_encoded = test_x_encoded[selected_features]
# for col in selected_features:
for col in feature_order:
    test_x_encoded[col] = test_x_encoded[col].astype(train_x_encoded[col].dtype)

In [30]:
train_x_encoded.shape

(120000, 68)

In [31]:
test_x_encoded.shape

(120000, 68)

In [32]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# import shap
# # 학습 및 테스트 데이터로 분할
# X_train, X_test, y_train, y_test = train_test_split(train_x_encoded, train_y_encoded, test_size=0.2, random_state=42)

# # XGBoost Classifier 모델 생성 및 학습
# model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
# model.fit(X_train, y_train)

# # 테스트 데이터로 예측
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.2f}")

# # SHAP 값 계산
# explainer = shap.Explainer(model, X_train)
# shap_values = explainer(X_test)

In [33]:
train_x_encoded['Fraud_Type'] = train_y_encoded

In [34]:
# 비율 조정을 위한 타겟 클래스별 비율 딕셔너리 (예: Normal 비율 1.0, 다른 클래스는 원하는 비율로 조정)
target_ratios = {
    0: 1.0,
    1: 1.0,
    2: 1.0,
    3: 1.0,
    4: 1.0,
    5: 1.0,
    6: 1.0,
    7: 1.0,
    8: 1.0,
    9: 1.0,
    10: 1.0,
    11: 1.0,
    12: 0.001,
}

# 각 클래스별로 샘플링하여 새로운 데이터프레임 생성
df_list = []
for target_class, ratio in target_ratios.items():
    df_class = train_x_encoded[train_x_encoded['Fraud_Type'] == target_class]
    num_class = len(df_class)
    
    # 비율에 맞게 샘플링
    df_sampled = df_class.sample(
        n=int(num_class * ratio), replace=False, random_state=42
    )
    
    # 리스트에 추가
    df_list.append(df_sampled)

# 샘플링된 데이터프레임 결합
df_concat = pd.concat(df_list, axis=0).reset_index(drop=True)

# 새로운 데이터프레임의 클래스별 카운트를 확인
print(df_concat.value_counts('Fraud_Type'))

Fraud_Type
12    118
0     100
1     100
2     100
3     100
4     100
5     100
6     100
7     100
8     100
9     100
10    100
11    100
dtype: int64


In [35]:
train_x_encoded_down = df_concat.drop(columns=['Fraud_Type'])
train_y_encoded_down = df_concat['Fraud_Type']

In [36]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier

catboost_model = CatBoostClassifier(verbose=0, random_state=42)
xgboost_model = XGBClassifier(eval_metric='logloss', random_state=42)
randomforest_model = RandomForestClassifier(random_state=42)

# 앙상블 모델 정의 (VotingClassifier)
voting_clf = VotingClassifier(
    estimators=[
        ('catboost', catboost_model),
        ('xgboost', xgboost_model),
        ('randomforest', randomforest_model)
    ],
    voting='soft'
)

# 앙상블 모델 학습
voting_clf.fit(train_x_encoded_down, train_y_encoded_down)

VotingClassifier(estimators=[('catboost',
                              <catboost.core.CatBoostClassifier object at 0x000001C30725DE50>),
                             ('xgboost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None, gamma=...
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('randomforest',
                              RandomForestClassifier(random_state=42))],
                 voting='soft')

In [37]:
# 예측
predictions = voting_clf.predict(test_x_encoded)
predictions_label = le_subclass.inverse_transform(predictions)

# Submission

In [38]:
# 분류 예측 결과 제출 데이터프레임(DataFrame)
# 분류 예측 결과 데이터프레임 파일명을 반드시 clf_submission.csv 로 지정해야합니다.
clf_submission = pd.read_csv("sample_submission.csv")
clf_submission["Fraud_Type"] = predictions_label
clf_submission.head()

,ID,Fraud_Type
0,TEST_000000,b
1,TEST_000001,m
2,TEST_000002,m
3,TEST_000003,m
4,TEST_000004,h


In [39]:
# 합성 데이터 생성 결과 제출 데이터프레임(DataFrame)
# 합성 데이터 생성 결과 데이터프레임 파일명을 반드시 syn_submission.csv 로 지정해야합니다.
all_synthetic_data.head()

,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,Customer_rooting_jailbreak_indicator,Customer_mobile_roaming_indicator,Customer_VPN_Indicator,Customer_loan_type,Customer_flag_terminal_malicious_behavior_1,Customer_flag_terminal_malicious_behavior_2,Customer_flag_terminal_malicious_behavior_3,Customer_flag_terminal_malicious_behavior_4,Customer_flag_terminal_malicious_behavior_5,Customer_flag_terminal_malicious_behavior_6,Customer_inquery_atm_limit,Customer_increase_atm_limit,Account_account_number,Account_account_type,Account_creation_datetime,Account_initial_balance,Account_balance,Account_indicator_release_limit_excess,Account_amount_daily_limit,Account_indicator_Openbanking,Account_remaining_amount_daily_limit_exceeded,Account_release_suspention,Account_one_month_max_amount,Account_one_month_std_dev,Account_dawn_one_month_max_amount,Account_dawn_one_month_std_dev,Transaction_Datetime,Transaction_Amount,Channel,Operating_System,Error_Code,Transaction_Failure_Status,Type_General_Automatic,IP_Address,MAC_Address,Access_Medium,Location,Recipient_Account_Number,Transaction_num_connection_failure,Another_Person_Account,Distance,Unused_terminal_status,Last_atm_transaction_datetime,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Fraud_Type,Transaction_resumed_date,Time_difference
0,1954,male,이선영,dhITBu-DbhAPPn,2012-12-10 22:02:43,D,0,1,1,0,0,0,0,c,0,0,0,0,0,0,0,1,aPjRmtBcgw,d,2011-04-07 23:06:07,157297,91118349,0,1000000,1,6563337,1,77469690,9695684,397823,949605,2016-07-21 06:11:30,11971340,Others,Others,a,0,automatic,123.119.143.95,82:a7:32:c6:40:99,a,경기도 하남시 천현동 37.531837 127.214564,kqPsmvZukj,0,1,11.894323,1,2007-12-08 22:23:59,2004-07-10 20:11:30,0,1,1,0,0,0,m,2010-06-24 05:20:33,7 days 22:50:08
1,1962,male,이재현,VUdWiC-wXhKmwF,2011-08-14 06:18:12,A,1,1,0,1,0,0,0,a,0,0,0,0,0,0,1,1,glGbFuBSkQ,a,2012-02-22 08:53:25,4949610,163179307,1,2000000,1,50409508,1,6905958,0,4013837,0,2032-08-21 18:03:24,-4985531,mobile,Android,a,0,automatic,117.88.205.134,1e:cb:ed:ec:5e:b1,a,전라북도 전주시 덕진구 산정동 35.840703 127.172249,OmCGgBfDJP,0,1,248.449917,1,2020-12-04 09:31:18,2014-09-27 19:01:36,0,0,0,0,0,0,m,2017-03-14 14:55:31,15 days 03:08:15
2,1978,male,최유진,UWTuyh-pgXndzV,2011-11-05 00:13:19,A,1,1,1,1,0,0,0,c,0,0,0,0,1,0,1,1,krDPXpovdl,b,2009-01-28 10:57:18,10851811,19555803,1,2000000,1,24627459,1,160624248,0,3825207,0,2034-09-03 21:39:20,-9021065,mobile,iOS,a,0,general,55.10.230.232,58:74:f3:e4:89:b1,a,경기도 동두천시 보산동 37.916631 127.05865,tVnckJaOJk,0,1,87.112229,1,2032-03-30 06:07:35,2004-07-10 20:11:30,0,0,0,0,0,0,m,2018-04-25 10:40:41,0 days 00:01:38
3,1978,male,이민재,BDBAtF-ZmBUHYl,2012-12-10 22:02:43,B,1,1,0,1,0,0,0,a,0,0,0,0,0,1,0,0,VJNRoGjnVl,c,2011-11-19 21:55:12,2200468,20043208,0,50000000,1,3118969,1,0,14338743,0,2444108,2009-09-20 00:24:55,-3444345,internet,Android,a,0,automatic,68.205.87.169,6a:bb:3a:1e:5a:a4,b,광주광역시 광산구 임곡동 35.219424 126.74449,wWuCXWJcYS,0,1,62.311807,1,2011-02-07 05:37:54,2009-12-12 18:38:03,0,0,1,0,0,0,m,2004-07-10 20:11:30,0 days 00:01:38
4,1990,male,김영환,UyFsAc-KBfHGXe,2010-08-03 07:30:55,B,1,1,1,0,1,0,0,a,1,0,0,0,0,0,1,1,JBPQMFLsdu,d,2010-11-25 10:09:43,12673470,24325340,0,2000000,1,6406882,1,0,0,0,3284366,2013-04-22 10:09:09,-3188083,mobile,Android,a,0,general,144.43.137.21,e2:1d:3e:be:5d:89,a,서울특별시 영등포구 양평동1가 37.523571 126.888814,TjTLsNbTyx,2,1,188.867984,0,2013-12-15 01:42:07,2029-06-03 22:15:02,0,1,1,0,3,0,m,2020-11-20 03:03:53,30 days 20:45:45


In [40]:
'''
(*) 저장 시 각 파일명을 반드시 확인해주세요.
    1. 분류 예측 결과 데이터프레임 파일명 = clf_submission.csv
    2. 합성 데이터 생성 결과 데이터프레임 파일명 = syn_submission.csv

(*) 제출 파일(zip) 내에 두 개의 데이터프레임이 각각 위의 파일명으로 반드시 존재해야합니다.
(*) 파일명을 일치시키지 않으면 채점이 불가능합니다.
'''
from datetime import datetime
today_datetime = datetime.today().strftime('%y%m%d_%H%M')
os.chdir('G:/내 드라이브/DACON_proj/DACON/2024_FSI_AIxData_Challenge')
# 폴더 생성 및 작업 디렉토리 변경
os.makedirs('./submission', exist_ok=True)
os.chdir('./submission')

# CSV 파일로 저장
clf_submission.to_csv('./clf_submission.csv', encoding='UTF-8-sig', index=False)
all_synthetic_data.to_csv('./syn_submission.csv', encoding='UTF-8-sig', index=False)

# ZIP 파일 생성 및 CSV 파일 추가
with zipfile.ZipFile(f'submission_{today_datetime}.zip', 'w') as submission:
    submission.write('clf_submission.csv')
    submission.write('syn_submission.csv')
    
print('Done.')
os.chdir('G:/내 드라이브/DACON_proj/DACON/2024_FSI_AIxData_Challenge')

Done.


In [41]:
import winsound

# 주파수와 지속시간 설정 (주파수 단위: Hertz, 지속시간 단위: 밀리초)
frequency = 1000  # 주파수 (Hertz)
duration = 500    # 지속시간 (Milliseconds)

# 소리 재생
winsound.Beep(frequency, duration)